# ETL para o banco de dados Neo4j

## Importações

In [1]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

## Constantes

In [2]:
DATASET_PATH = '../../dataset'

USER_FILTERED_PATH = os.path.join(DATASET_PATH, 'user-filtered.csv')
ANIME_FILTERED_PATH = os.path.join(DATASET_PATH, 'anime-filtered.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Configurando inicialização do "conector" Neo4j spark

In [4]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-neo4j/neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar").\
        getOrCreate()

## Lendo o arquivo user-filtered.csv

In [5]:
df_user_filtred = spark.read.csv(USER_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [6]:
df_user_filtred.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      67|     9|
|      0|    6702|     7|
|      0|     242|    10|
|      0|    4898|     0|
|      0|      21|    10|
|      0|      24|     9|
|      0|    2104|     0|
|      0|    4722|     8|
|      0|    6098|     6|
|      0|    3125|     9|
|      0|     481|    10|
|      0|      68|     6|
|      0|    1689|     6|
|      0|    2913|     6|
|      0|    1250|     7|
|      0|     356|     9|
|      0|     121|     9|
|      0|     430|     9|
|      0|    1829|     7|
|      0|    1571|    10|
+-------+--------+------+
only showing top 20 rows



## Filtrando a base de dados

Para tentar resolver isso, filtramos a base de dados, removendo todas as avaliações abaixo de 8. Como definimos o limiar para 8, os resultados das consultas continuarão iguais. Porém, seria ideal armazenar todas as relações, pois caso esse limiar seja modificado para algum valor inferior, seria necessário adicionar no banco de dados as interações com as notas inferiores. Caso todas as relações fossem armazenadas, bastaria modificar a filtragem para considerar esse novo valor de limiar. Além disso, caso o sistema de recomendações fosse modificado, para usar tanto as avaliações negativas como negativas para gerar recomendações, seria necessário ter todas as avaliações no banco de dados.

In [7]:
df_user_filtred = df_user_filtred.filter(df_user_filtred.rating == 10)
display(df_user_filtred.show())
df_user_filtred.count()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     242|    10|
|      0|      21|    10|
|      0|     481|    10|
|      0|    1571|    10|
|      0|     578|    10|
|      0|    2236|    10|
|      0|     415|    10|
|      0|     235|    10|
|      1|   40028|    10|
|      1|    1575|    10|
|      1|    5114|    10|
|      1|   32281|    10|
|      1|      20|    10|
|      1|    1735|    10|
|      1|   37450|    10|
|      1|     199|    10|
|      1|   16498|    10|
|      1|   25777|    10|
|      1|   35760|    10|
|      1|    9253|    10|
+-------+--------+------+
only showing top 20 rows



None

7144392

In [8]:
# df_user_filtred = df_user_filtred.limit(1000000)

## Lendo o arquivo anime_filtered.csv

In [9]:
df_anime_filtred = spark.read.csv(ANIME_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)
df_anime_filtred = df_anime_filtred.select(['anime_id', 'name'])

## Fazendo o join nos animes

In [10]:
df_animes = df_user_filtred.select('anime_id').distinct().join(df_anime_filtred, 'anime_id').select(['anime_id', 'name'])

## Adicionando uma coluna "img" nos animes

In [11]:
df_animes = df_animes.withColumn("img", lit('img-path'))
df_animes.show()

+--------+--------------------+--------+
|anime_id|                name|     img|
+--------+--------------------+--------+
|    8086|Densetsu no Yuush...|img-path|
|   30654|Ansatsu Kyoushits...|img-path|
|   40515|Nihon Chinbotsu 2020|img-path|
|    5300|Zoku Natsume Yuuj...|img-path|
|   22097|Magi: Sinbad no B...|img-path|
|   33569|Re:Petit kara Haj...|img-path|
|     148|Kita e.: Diamond ...|img-path|
|   25517|    Magic Kaito 1412|img-path|
|    6336|Mobile Suit Gunda...|img-path|
|   17389|  Kingdom 2nd Season|img-path|
|    2866|         Ane☆Haramix|img-path|
|    9465|Break Blade 4: Sa...|img-path|
|   38422|High Score Girl: ...|img-path|
|    6658|Unko-san: Tsuiter...|img-path|
|    1088|             Macross|img-path|
|    6654|     Namakura Gatana|img-path|
|    1829|           Ged Senki|img-path|
|   34239|Mutsugo to Ouma n...|img-path|
|   11033|Natsu-iro Egao de...|img-path|
|    3918|         Resort Boin|img-path|
+--------+--------------------+--------+
only showing top

## Salvando os dados no banco de dados

### Salvando os nós de usuários

In [12]:
display(df_user_filtred.select('user_id').distinct().show())
df_user_filtred.select('user_id').distinct().count()

+-------+
|user_id|
+-------+
|    148|
|    463|
|    496|
|    833|
|   1088|
|   1342|
|   1580|
|   1591|
|   1645|
|   1959|
|   2142|
|   2366|
|   2659|
|   2866|
|   3175|
|   3749|
|   3794|
|   3997|
|   4101|
|   4519|
+-------+
only showing top 20 rows



None

295741

In [13]:
df_user_filtred.select('user_id').distinct().write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":User") \
  .option("node.keys", "user_id") \
  .option("schema.optimization.type", "INDEX") \
  .save()

### Salvando os nós de animes

In [14]:
display(df_animes.show())
df_animes.count()

+--------+--------------------+--------+
|anime_id|                name|     img|
+--------+--------------------+--------+
|    8086|Densetsu no Yuush...|img-path|
|   30654|Ansatsu Kyoushits...|img-path|
|   40515|Nihon Chinbotsu 2020|img-path|
|    5300|Zoku Natsume Yuuj...|img-path|
|   22097|Magi: Sinbad no B...|img-path|
|   33569|Re:Petit kara Haj...|img-path|
|     148|Kita e.: Diamond ...|img-path|
|   25517|    Magic Kaito 1412|img-path|
|    6336|Mobile Suit Gunda...|img-path|
|   17389|  Kingdom 2nd Season|img-path|
|    2866|         Ane☆Haramix|img-path|
|    9465|Break Blade 4: Sa...|img-path|
|   38422|High Score Girl: ...|img-path|
|    6658|Unko-san: Tsuiter...|img-path|
|    1088|             Macross|img-path|
|    6654|     Namakura Gatana|img-path|
|    1829|           Ged Senki|img-path|
|   34239|Mutsugo to Ouma n...|img-path|
|   11033|Natsu-iro Egao de...|img-path|
|    3918|         Resort Boin|img-path|
+--------+--------------------+--------+
only showing top

None

14540

In [15]:
df_animes.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":Anime") \
  .option("node.keys", "anime_id,name,img") \
  .save()

CRIAR INDICE UTILIZANDO A [INTERFACE](http://localhost:7474)

`CREATE INDEX anime_id_index FOR (anime: Anime) ON anime.anime_id`

### Salvando as relações entre usuários e animes

In [16]:
df_user_filtred.write.format("org.neo4j.spark.DataSource")\
    .mode("Append")\
    .option("url", "neo4j://localhost:7687")\
    .option("relationship", "Rating")\
    .option("relationship.save.strategy", "keys")\
    .option("relationship.source.labels", ":User")\
    .option("relationship.source.save.mode", "match")\
    .option("relationship.source.node.keys", "user_id:user_id")\
    .option("relationship.target.labels", ":Anime")\
    .option("relationship.target.save.mode", "match")\
    .option("relationship.target.node.keys", "anime_id:anime_id")\
    .option("relationship.properties", "rating")\
    .save()